In [13]:
!pip install groq pydantic python-dotenv pandas

A Weaker model is selected intentionally becuase incase of a more capable model its harder to notice the impacts of prompt

In [14]:
import os 
from dotenv import load_dotenv
from groq import Groq

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
llm=Groq(api_key=GROQ_API_KEY)

MODEL_NAME = "openai/gpt-oss-20b"   #weak model
TEMPERATURE = 0



In [15]:
def call_groq(prompt):
    response = llm.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=TEMPERATURE
    )
    return response.choices[0].message.content


In [16]:
from pydantic import BaseModel, Field
from typing import List, Optional

class ExtractedItem(BaseModel):
    value: str = Field(..., description="Exact text from input")
    description: Optional[str] = Field(
        None, description="Only if explicitly stated"
    )

class ExtractionResult(BaseModel):
    items: List[ExtractedItem]


In first prompt we have left out additional context of what exact indian food and not stated partiuclar rules of what we exactly want.

In [17]:
prompt_v1 = """
Extract all important items from the following text and give me instructions to make indian food.

Text:
Ingredients: Flour,sugar,salt,oil,plastic bag,spoon,tissue,olive,ranch
"""

output_v1 = call_groq(prompt_v1)
print(output_v1)


**Important items extracted from the text**

| # | Item |
|---|------|
| 1 | Flour |
| 2 | Sugar |
| 3 | Salt |
| 4 | Oil |
| 5 | Plastic bag |
| 6 | Spoon |
| 7 | Tissue |
| 8 | Olive |
| 9 | Ranch |

> **Note:**  
> - “Olive” likely refers to *olive oil*.  
> - “Ranch” is a salad‑dressing mix and isn’t used in traditional Indian cooking.  
> - The plastic bag, spoon, and tissue are kitchen tools, not ingredients.

---

## Quick & Easy Indian Sweet: **Mysore Pak**  
*(A classic South‑Indian sweet that uses only the ingredients you have. Olive oil can replace ghee if you prefer.)*

### Ingredients
- 1 cup all‑purpose flour (or whole‑wheat flour for a healthier version)  
- 1 cup sugar (adjust to taste)  
- 1/2 cup oil (olive oil works fine)  
- 1/2 tsp salt (optional, for balance)  
- 1/4 tsp cardamom powder (optional, for aroma)  

### Equipment
- Medium‑sized saucepan  
- Wooden spoon or spatula  
- Plastic bag (to store the dough, if you want to make it ahead)  
- Tissue or clean ki

In the second version of the prompt we have assigned the added context of which food and rules of what exactly the user wants.

In [39]:
prompt_v2 = """
Extract all important items from the following text and give me instructions to make bhature.

Rules:
- Use only items explicitly present
- Important items are the items which are relevant to the recipe
- Make sure the instructions are point wise
- Mention the temperature if needed,quantity if needed
- Use only important items 

Text:
Ingredients: Flour,sugar,salt,oil,plastic bag,spoon,tissue,olive,ranch
"""

output_v2 = call_groq(prompt_v2)
print(output_v2)


**Important items used for bhature**  
- Flour  
- Salt  
- Oil  

**Instructions**

1. **Prepare the dough**  
   - In a bowl, combine the flour and salt.  
   - Gradually add water, stirring until a smooth, pliable dough forms.  
   - Knead the dough for a few minutes until elastic.  
   - Cover the dough and let it rest for at least 30 minutes.

2. **Shape the bhatures**  
   - Divide the rested dough into equal portions.  
   - Roll each portion into a smooth ball, then flatten it into a disc about 1 cm thick.

3. **Heat the oil**  
   - Pour oil into a deep pan or kadai.  
   - Heat the oil until it is hot enough to fry the bhatures (the dough should sizzle and rise to the surface quickly).

4. **Fry the bhatures**  
   - Gently slide a flattened dough disc into the hot oil.  
   - Fry until the bhature puffs up and turns golden brown on both sides, turning it with a slotted spoon.  
   - Remove the bhature and drain on paper towels.

5. **Serve**  
   - Serve the hot bhatures wit

In the Third Version we have added elements of role prompting by assigning role to the llm, In turn it tried to explain the process of making it as if its someone who is new to cooking.

In [19]:
prompt_v3 = """
System:
You are an expert chef whose role is to teach beginners how to cook in a detailed,
step-by-step manner.

User:
Your task is to extract all important items from the given text and provide
point-wise instructions to make bhature.

Rules:
- Use only items explicitly present in the text
- Important items are those relevant to the recipe
- Instructions must be point-wise
- Mention temperature and quantity if needed
- Use only important items

Text:
Ingredients: Flour, sugar, salt, oil, plastic bag, spoon, tissue, olive, ranch
"""


output_v3 = call_groq(prompt_v3)
print(output_v3)


**Step‑by‑step instructions to make bhature (using only the items listed in the text)**  

1. **Prepare the dough**  
   - In a clean plastic bag, add **2 cups of flour**, **1 tsp sugar**, **1 tsp salt**, and **1 tbsp oil**.  
   - Seal the bag and shake vigorously until the mixture is uniform.  
   - Open the bag, transfer the dough onto a lightly oiled surface, and knead with a spoon (or your hands) for **5–7 minutes** until smooth and elastic.  

2. **Rest the dough**  
   - Cover the dough with a clean tissue to keep it from drying out.  
   - Let it rest at room temperature for **30 minutes**.  

3. **Shape the bhatures**  
   - Divide the rested dough into 6–8 equal portions.  
   - Roll each portion into a flat circle about **6–8 inches** in diameter.  

4. **Heat the oil**  
   - Pour enough **olive oil** into a deep‑fat pan to submerge the bhatures.  
   - Heat the oil to **180 °C (350 °F)**.  

5. **Fry the bhatures**  
   - Carefully slide one bhature into the hot oil.  
   

The "a" iteration of fourth version of the prompt is suppsoed to show that if adequate information is not provided as per the requirements of the prompt attaining no hallucination is impossible because the model refuses to generate futher without factual information. 

In [38]:
prompt_v4a = """
System:
You are a strict information extraction assistant.
You must NOT use world knowledge, assumptions, or inference.

User:
Your task is to extract items explicitly mentioned in the text relevant to making a bhature.

Rules:
- Extract ONLY items explicitly present in the text
- Do NOT generate recipes or cooking instructions
- Do NOT infer relevance beyond the text
- Do NOT guess quantities, temperatures, or steps
- If the text does NOT contain enough information to perform a task,
  clearly state that the task cannot be completed 

Output format:
- Return a bullet list of extracted items
- If no valid items exist, return an empty list

Text:
Ingredients: Flour, sugar, salt, oil, plastic bag, spoon, tissue, olive, ranch
"""

output_v4a = call_groq(prompt_v4a)
print(output_v4a)


The task cannot be completed.


In the "b" iteration of the fourth version we have allowed "world knowledge" at the risk of hallucination the balance of this is needed for least amount of halliucination and some important wants.

In [21]:
prompt_v4b = """
System:
You are an expert chef.

User:
Your task is to identify which items from the text are typically used
to make bhature, using common culinary knowledge.

Rules:
- Use culinary knowledge to assess relevance
- Only select items that are typically used for bhature
- Clearly ignore non-relevant items
- Do not invent ingredients not present in the text
- Do not include quantities or temperatures

Output format:
- Bullet list of relevant items
- Brief justification (one line each)

Text:
Ingredients: Flour, sugar, salt, oil, plastic bag, spoon, tissue, olive, ranch
"""

output_v4b=call_groq(prompt_v4b)
print(output_v4b)

- **Flour** – The main ingredient for the dough.  
- **Sugar** – A small amount is often added to aid fermentation and flavor.  
- **Salt** – Essential for seasoning the dough.  
- **Oil** – Used both in the dough and for deep‑frying the bhature.


In the fifth version we properly role prompt by making sure the models behaviour remains consistent across different queries we observe that the model is able to performs better in terms of classification and the responses have better accuracy while being concise.

In [34]:
SYSTEM_PROMPT = """
You are an expert Chef with a lot of expereince in teaching how to cook.

Behavior rules:
- Do not invent ingredients that are not explicitly provided
- Use culinary knowledge ONLY when the user explicitly asks for relevance
- Do not guess quantities, temperatures, or cooking steps unless provided
- Ignore irrelevant or non-food items
- Be concise, structured, and factual
"""



In [35]:
def call_groq(prompt):
    response = llm.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=TEMPERATURE
    )
    return response.choices[0].message.content


In [ ]:
prompt_v5 = """
Your task is to extract all important items from the given text and provide
instructions to make bhature.
Text:
Ingredients: Flour, sugar, salt, oil, plastic bag, spoon, tissue, olive, ranch
"""

print(call_groq(prompt_v5))




**Key items from the text**

| Category | Items |
|----------|-------|
| **Ingredients** | Flour, salt, oil |
| **Utensils / Misc.** | Plastic bag, spoon, tissue |
| **Not needed for bhature** | Sugar, olive, ranch |

---

### Simple bhature recipe (using only the provided items)

1. **Prepare the dough**  
   - In a bowl, combine **flour** and **salt**.  
   - Add **oil** and mix with a **spoon** until the mixture resembles coarse crumbs.  
   - Gradually add water (not listed but required for dough) until a soft, pliable dough forms.  
   - Knead the dough until smooth.

2. **Rest the dough**  
   - Wrap the dough in a **plastic bag** or cover it with a damp **tissue** to keep it from drying out.  
   - Let it rest for a short period (time not specified).

3. **Shape and cook**  
   - Divide the dough into small portions.  
   - Roll each portion into a flat disc.  
   - Heat **oil** in a pan and fry the discs until golden brown on both sides.

4. **Serve**  
   - Remove the bhatures